In [ ]:
import numpy as np
from dataloader import SimulationData
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import cv2

size = 250

train_transform = transforms.Compose([transforms.ToTensor(),
                                #transforms.Resize((size,size)),
                                #transforms.CenterCrop((1200,1000))
                                ])
val_transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Resize((size,size)),
                                #transforms.Grayscale(num_output_channels = 1),
                                ])

repair_type = '001_hansson_pin_system'
data_dir = '/work3/dgro/Data/'
batch_size = 16

dataset = SimulationData(repair_type = repair_type, split = 'train', data_path = data_dir, transform = train_transform)

In [ ]:
idx = 10

images, scores = dataset.__getitem__(idx)

im_frontal = images.permute(1, 2, 3, 0)[0]
im_lateral= images.permute(1, 2, 3, 0)[1]

gray0 = cv2.cvtColor(im_frontal, cv2.COLOR_BGR2GRAY)

plt.imshow(im_frontal)
print(im_frontal.shape)

In [ ]:
im_frontal_plain = cv2.imread('/work3/dgro/frontal.jpg') 
gray1 = cv2.cvtColor(im_frontal_plain, cv2.COLOR_BGR2GRAY)
plt.imshow(gray1)
print(gray1.shape)

In [ ]:
#keypoints
sift = cv2.xfeatures2d.SIFT_create()
keypoints_1, descriptors_1 = sift.detectAndCompute(im_frontal_plain,None)

img_1 = cv2.drawKeypoints(gray1,keypoints_1,im_frontal_plain)
plt.imshow(img_1)